In [ ]:
from langchain_groq import ChatGroq

from langgraph.graph import StateGraph,START,END

from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from pydantic import BaseModel,Field
from typing import Annotated
from langgraph.graph.message import BaseMessage


In [ ]:
class jobData(BaseModel):
    instrctions : str = Field(default= "you are an expert in writing short and consice email with ", description="Instruction for creating a proper message")
    subject : str = Field(default="")
    body : str  = Field(default="")
    email :str = Field(default="")
    post_data :str = Field(default="")



In [3]:
model = ChatGroq(model= "openai/gpt-oss-20b")

In [ ]:
# email extractor
def extract_mail_id(state:jobData):
    prompt = PromptTemplate(
        template= "From the text below, extract ONLY the email address  where the candidate has to apply.  Return ONLY the email (no other text).\n\n Post content:\n{post_data}"
    )
    mail_chain = prompt | model | StrOutputParser

    email = mail_chain.invoke({'post_chain':state.post_data})

    return {'email':email}

In [ ]:
def create_mail(state)